In [ ]:
import pandas as pd
from pathlib import Path
import json
from IPython.display import display, Markdown
from sklearn.metrics import PrecisionRecallDisplay

In [ ]:
metrics_files = list(Path('.').glob('*json'))
model_names_dict = {'13z30zpq':'without_3', '3uh7ste9':'without_6_bg', 
                    '369fkgdi': 'all_together', '2hwh4w5d':'without_6', 
                    '37ftgpl8':'without_7_11_12', '2qg7u58e':'all_together_bg', 
                    '20pp7z59':'without_3_bg'}

In [ ]:
dicts_list = []
probs_dict = {}
for file in metrics_files:
    with open(file, 'r') as f:
        metrics = json.load(f)
    metrics['dataset'] = file.stem.split('-')[-1]
    metrics['model'] = model_names_dict[file.stem.split('-')[-2]]
    metrics['filename'] = file.stem
    probs_dict[file.stem] = pd.read_csv(file.with_suffix('.csv'))
    dicts_list.append(metrics)

In [ ]:
all_results = pd.DataFrame(dicts_list)
all_results = all_results[['model', 'accuracy', 'f1score', 'precision', 'recall', 'auc', 'dataset', 'filename']]

In [ ]:
available_datasets = all_results.dataset.unique()

In [ ]:
def plot_recall_precision(df_row, probs_dict):
    prob_dict = probs_dict[df_row['filename']]
    PrecisionRecallDisplay.from_predictions(prob_dict['label'], prob_dict['class1_prob'], 
                                            name=df_row['model'])

def plot_results_for_dataset(dataset_name, all_results_df, probs_dict):
    display(Markdown(dataset_name))
    current_results = all_results[all_results['dataset']== dataset_name]
    current_results = current_results.sort_values('auc')
    display(current_results.drop(columns=['dataset', 'filename']))
    for _, item in current_results.iterrows():
        plot_recall_precision(item, probs_dict)    

In [ ]:
plot_results_for_dataset(available_datasets[0], all_results, probs_dict)

In [ ]:
plot_results_for_dataset(available_datasets[1], all_results, probs_dict)

In [ ]:
plot_results_for_dataset(available_datasets[2], all_results, probs_dict)

In [ ]:
plot_results_for_dataset(available_datasets[3], all_results, probs_dict)